In [1]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import google.generativeai as genai
import plotly.graph_objects as go

/home/zairo-bastos/Documentos/transportes/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
mobilidade = pd.read_csv("./data/dados_onibus.csv", delimiter=',', encoding='UTF-8', low_memory=False)
mobilidade

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes,domingo,segunda,terca,quarta,quinta,sexta,sabado,feriado,vespera_feriado
0,1,2018-01-01 00:00:00,29,0,0.000000e+00,1.000000,0,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,1,2018-01-01 01:00:00,58,0,2.697968e-01,0.962917,1,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,1,2018-01-01 02:00:00,42,0,5.195840e-01,0.854419,2,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,1,2018-01-01 03:00:00,49,0,7.308360e-01,0.682553,3,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,1,2018-01-01 04:00:00,112,0,8.878852e-01,0.460065,4,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150161,999,2018-07-02 21:00:00,5,0,-5.195840e-01,0.854419,21,2,183,7,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1150162,999,2018-07-02 22:00:00,8,0,-2.697968e-01,0.962917,22,2,183,7,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1150163,999,2018-07-02 23:00:00,1,0,-2.449294e-16,1.000000,23,2,183,7,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1150164,999,2018-07-30 20:00:00,2,0,-7.308360e-01,0.682553,20,30,211,7,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [11]:
top_linhas = [52, 41, 51, 24, 45, 76, 42, 26, 375, 28, 72, 711, 74, 43, 815, 372, 15, 38, 366, 29]

#### Prompt

In [7]:
def primeira_semana(df_selecionado):
    mapeamento_dias = {
        0: 'Segunda-feira',
        1: 'Terça-feira',
        2: 'Quarta-feira',
        3: 'Quinta-feira',
        4: 'Sexta-feira',
        5: 'Sábado',
        6: 'Domingo'
    }
    df_transport2 = df_selecionado.head(168)
    dias_da_semana = df_transport2['d_semana'].unique().tolist()        

    # Cria uma string formatada com os dias da semana e suas posições
    dias_encontrados = ""
    for i, dia in enumerate(dias_da_semana):
        pos_inicio = i * 24
        pos_fim = pos_inicio + 23
        dias_encontrados += f"  - Dia {i+1}: posições {pos_inicio} a {pos_fim} ({mapeamento_dias[dia]});\n"

    return dias_encontrados

In [8]:
def quantidade_dias(df_selecionado):
    shape = df_selecionado.shape[0]
    dias = shape // 24
    return dias

In [4]:
def calc_smape(resposta,passageiros):
    previsao = resposta
    tam = len(previsao)
    if tam > 168:
        tam = 168

    real = passageiros[:tam]
    previsto = previsao[:tam]

    real, previsto = np.array(real), np.array(previsto)

    numerador = np.abs(real - previsto)
    denominador = np.abs(real) + np.abs(previsto)

    denominador = [denominador[i] if denominador[i]!=0 else 1 for i in range(len(denominador))]
    denominador = np.array(denominador)

    smape = round(np.mean(numerador / (denominador/2))*100, 2)
    return smape,real,previsto

In [ ]:
data_inicio='2018-03-01'
data_fim='2018-06-04'

#configuração da API
variables = load_dotenv()
variables = os.getenv('API_KEY')

genai.configure(api_key=variables)

#configuração do modelo
generation_config ={
    "candidate_count": 1,
    "temperature": 1.0,
}

#mapeamento_dias
mapeamento_dias = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

for linha in top_linhas:
    # Seleciona a linha
    mobilidade2 = mobilidade[mobilidade['linha'] == linha]

    #seleção de colunas
    mobilidade2 = mobilidade2[['data_hora','d_semana','validations_per_hour', 'feriado','vespera_feriado']].copy()
    mobilidade2['data_hora'] = pd.to_datetime(mobilidade2['data_hora'])

    #removendo os duplicados
    mobilidade2['validations_per_hour'] = mobilidade2.groupby('data_hora')['validations_per_hour'].transform('sum')
    mobilidade2 = mobilidade2.drop_duplicates(keep='last')

    #criando um dataframe com todas as datas
    datas = []
    for mes in range(1,8):
        for dia in range(1, 29 if mes == 2 else 31 if mes in [4, 6] else 32):
            for hora in range(24):
                datas.append(pd.Timestamp(2018, mes, dia, hora))

    df_completo = pd.DataFrame({'data_hora': datas})

    # selecionando os feriados e vésperas de feriados
    feriados = mobilidade2[mobilidade2['feriado'] == 1]['data_hora'].dt.date.unique().tolist()
    vespera_feriados = mobilidade2[mobilidade2['vespera_feriado'] == 1]['data_hora'].dt.date.unique().tolist()

    #criando um novo dataframe com as informações de mobilidade
    df_transport2 = pd.merge(df_completo, mobilidade2, on='data_hora', how='left').fillna(0)
    df_transport2['d_semana'] = df_transport2['data_hora'].dt.dayofweek
    df_transport2['feriado'] = df_transport2['data_hora'].dt.date.isin(feriados).astype(int)
    df_transport2['vespera_feriado'] = df_transport2['data_hora'].dt.date.isin(vespera_feriados).astype(int)

    df_transport2['validations_per_hour'] = df_transport2['validations_per_hour'].astype(int)
    df_transport2.reset_index(drop=True, inplace=True) 

    #selecionando o período de análise
    data_inicio = pd.to_datetime(data_inicio)
    data_fim = pd.to_datetime(data_fim)

    #selecionando os dados
    df_selecionado = df_transport2[(df_transport2['data_hora'] >= data_inicio) & (df_transport2['data_hora'] < data_fim)]
    passageiros = df_selecionado['validations_per_hour'].values.tolist()

    df_exato = df_transport2[df_transport2['data_hora'] >= data_fim].head(168)
    exato = df_exato['validations_per_hour'].values.tolist()

    #Feriados
    dataset_feriados = df_selecionado.reset_index(drop=True)
    dataset_feriados2 = dataset_feriados[dataset_feriados['feriado'] == 1]

    list_datas_feriados = dataset_feriados2['data_hora'].dt.date.unique().tolist()
    dias_feriados = ""
    for data in list_datas_feriados:
        indices_dia = dataset_feriados2[dataset_feriados2['data_hora'].dt.date == data].index.tolist()
        dia = indices_dia[0] // 24 + 1  # Calcula o número do dia
        dias_feriados += f"  - Dia {dia}: posições {indices_dia[0]} a {indices_dia[-1]} ({mapeamento_dias[data.weekday()]});\n"

    #Próximos Dias
    list_datas_proximos_dias = df_exato['data_hora'].dt.date.unique().tolist()
    dias_proximos = ""
    for i, data in enumerate(list_datas_proximos_dias):
        pos_inicio = i * 24
        pos_fim = pos_inicio + 23
        dia_util = "Dia útil" if data.weekday() < 5 else "Final de semana"
        if df_exato[df_exato['data_hora'].dt.date == data]['feriado'].values[0] == 1:
            dia_util = "Feriado"
        dias_proximos += f"Dia {i+1} - {mapeamento_dias[data.weekday()]} (posições {pos_inicio}-{pos_fim} da sua previsão): {dia_util};\n"

    horas = 168
    prompt = f""" Você é um assistente de previsão de séries temporais encarregado de analisar dados de uma série temporal específica.
        
A série temporal tem dados de {quantidade_dias(df_selecionado)} período(s) consecutivos. Cada anotação da série temporal representa a incidência de um evento que ocorre a cada hora.

Por exemplo, um série temporal pode ser representado assim:
{passageiros[:24]}

Seu objetivo é prever a incidência de um evento para as próximas N horas, levando em consideração não apenas os períodos anteriores, mas também o contexto geral.

Para fazer isso com precisão, leve em consideração: 
- Padrões sazonais: Identifique picos em determinados períodos. 
- Variações de Padrões: Padrões podem variar de acordo com o dia da semana ou eventos especiais (feriados, fins de semana).

Regras da Saída:
Após analisar os dados fornecidos e compreender os padrões de tráfego, gere uma previsão para as próximas N horas, com as seguintes regras: 
- A saída deve ser uma lista contendo apenas os valores previstos, sem explicação adicional ou texto introdutório.
- Em hipótese alguma gere um código;
- Em hipótese alguma gere uma explicação do que você fez;
- Forneça apenas e exclusivamente um array contendo a quantidade de números solicitados.
- A previsão deve começar imediatamente após o último dado fornecido.

Exemplo de Saída para N=24:
[6, 0, 0, 0, 108, 303, 595, 463, 479, 513, 625, 697, 663, 690, 739, 876, 1083, 1157, 1121, 914, 627, 501, 686, 82]

Instruções Adicionais:
- Padrões Semanais: Utilize os dados fornecidos para entender padrões sazonais, como picos de incidência em determinados períodos.
- Eventos Especiais: A ocorrência de eventos é significativamente afetada por feriados e outros eventos importantes.
- Dia da Semana: O dia da semana também influencia a ocorrência de eventos. 
- Duração de um evento: A série temporal fornecida representa a ocorrência de um evento a cada hora.

Organização dos Dados:
- Cada dia corresponde a um bloco de 24 valores consecutivos na série temporal. Por exemplo:\n{primeira_semana(df_selecionado)}
- E assim por diante.
- A cada 24 valores, ocorre a transição para o próximo dia.
- Períodos em que temos feriado na série temporal:\n{dias_feriados}

Serie temporal a ser analisada:
{passageiros}

Contexto dos dias a serem previstos:\n{dias_proximos}
Gere um array contendo os próximos {horas} (N={horas}) números da sequência:
"""
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
    response = model.generate_content(prompt, generation_config=generation_config)
    resposta = response.text
    qtd_tokens = model.count_tokens(prompt)
    
    smape,real,previsto = calc_smape(resposta,exato)
        
    os.makedirs("./imgteste", exist_ok=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(real))), y=real, mode='lines', name='Valor Real'))
    fig.add_trace(go.Scatter(x=list(range(len(previsto))), y=previsto, mode='lines', name='Valor Previsto'))

    fig.update_layout(
        title=f'Passageiros por Horário - smape: {smape}%',
        xaxis_title='Horários',
        yaxis_title='Passageiros',
        showlegend=True,
        colorway=['#1f77b4', '#ff7f0e'],
        height=600,
    )
    fig.write_image(f"./imgteste/{linha}.png", width=1400, height=600)
    
    print(f"Imagem gerada para a linha {linha}")

In [9]:
linha = 52
resposta = [38.26368713378906,
     8.932841300964355,
     4.855633735656738,
     11.908289909362793,
     91.7135238647461,
     474.3111267089844,
     826.9880981445312,
     714.3662719726562,
     418.8068542480469,
     322.6533508300781,
     295.8226318359375,
     340.3727722167969,
     377.2383728027344,
     368.3013610839844,
     387.493896484375,
     455.92266845703125,
     575.6940307617188,
     690.5806884765625,
     553.5009155273438,
     347.7838439941406,
     238.7248077392578,
     228.094482421875,
     250.23968505859375,
     136.98284912109375,
     46.8959846496582,
     11.114356994628906,
     5.601412296295166,
     12.495000839233398,
     99.29862976074219,
     455.6257019042969,
     842.0734252929688,
     750.2882690429688,
     428.3658142089844,
     345.1449279785156,
     317.7079772949219,
     357.2451171875,
     392.1483459472656,
     375.4804992675781,
     380.1701354980469,
     445.87750244140625,
     617.0960083007812,
     703.4996948242188,
     584.3807983398438,
     360.5532531738281,
     245.02999877929688,
     229.84243774414062,
     238.12478637695312,
     128.1531982421875,
     44.273921966552734,
     10.811869621276855,
     5.166442394256592,
     11.149810791015625,
     84.79785919189453,
     417.5266418457031,
     802.7058715820312,
     799.3859252929688,
     494.15802001953125,
     377.1989440917969,
     343.6734924316406,
     379.0849914550781,
     388.43389892578125,
     358.4252014160156,
     362.39373779296875,
     419.8385009765625,
     504.2986755371094,
     563.356201171875,
     525.319580078125,
     393.680908203125,
     289.1435852050781,
     286.1332702636719,
     310.7359619140625,
     173.79115295410156,
     62.82947540283203,
     14.418060302734375,
     6.355432987213135,
     12.559700965881348,
     88.74366760253906,
     457.6202697753906,
     836.43310546875,
     821.6100463867188,
     642.838134765625,
     586.1642456054688,
     519.9326782226562,
     491.9027404785156,
     494.9141540527344,
     463.2094421386719,
     474.585693359375,
     579.0245361328125,
     735.9996337890625,
     883.8775634765625,
     752.6277465820312,
     565.402587890625,
     394.5435791015625,
     333.0809020996094,
     343.0080261230469,
     181.81382751464844,
     62.05534744262695,
     12.848090171813965,
     6.210016250610352,
     13.701199531555176,
     104.29685974121094,
     482.5415954589844,
     837.8417358398438,
     757.8052978515625,
     466.7908630371094,
     381.1904296875,
     340.2448425292969,
     382.80230712890625,
     409.3394775390625,
     390.1623229980469,
     403.7010498046875,
     482.45635986328125,
     648.5520629882812,
     723.8552856445312,
     592.491455078125,
     393.7012939453125,
     276.8707580566406,
     267.21319580078125,
     295.9745788574219,
     165.16453552246094,
     68.34479522705078,
     13.838436126708984,
     6.912282943725586,
     13.411674499511719,
     84.4327163696289,
     372.9418029785156,
     701.3958129882812,
     725.3908081054688,
     495.67169189453125,
     410.0955810546875,
     401.48223876953125,
     451.92669677734375,
     460.9444274902344,
     422.3707580566406,
     411.1027526855469,
     442.73980712890625,
     491.2923583984375,
     534.90673828125,
     505.7449951171875,
     408.58349609375,
     309.5494079589844,
     306.03173828125,
     355.95367431640625,
     202.3541717529297,
     81.46601867675781,
     14.158942222595215,
     6.400508403778076,
     12.22265338897705,
     83.32086944580078,
     424.3077392578125,
     801.21630859375,
     828.54931640625,
     687.6314086914062,
     722.0538940429688,
     731.9393920898438,
     705.4356079101562,
     697.8565673828125,
     672.9139404296875,
     669.7794189453125,
     681.3657836914062,
     875.3566284179688,
     1036.97314453125,
     978.7354125976562,
     791.5303955078125,
     656.8555297851562,
     525.2109985351562,
     332.8787841796875,
     178.44004821777344]
exato = [18, 0, 0, 0, 126, 516, 928, 872, 450, 382, 316, 383, 476, 446, 417, 486, 640, 667, 542, 409, 286, 271, 286, 139, 50, 0, 0, 0, 133, 488, 912, 808, 443, 387, 336, 415, 451, 337, 412, 425, 644, 799, 564, 405, 230, 276, 328, 125, 73, 0, 0, 0, 148, 460, 899, 811, 472, 390, 318, 447, 446, 385, 466, 435, 615, 725, 544, 369, 286, 260, 270, 139, 48, 0, 0, 0, 139, 467, 848, 758, 491, 396, 301, 450, 412, 452, 443, 600, 658, 687, 451, 432, 272, 251, 284, 168, 50, 0, 0, 0, 129, 458, 865, 746, 458, 385, 373, 396, 401, 433, 455, 608, 713, 666, 543, 377, 285, 288, 324, 164, 85, 0, 0, 0, 90, 443, 694, 744, 455, 428, 331, 504, 535, 457, 393, 427, 535, 437, 501, 454, 330, 334, 412, 293, 98, 0, 0, 0, 71, 523, 686, 887, 763, 794, 785, 710, 688, 640, 744, 813, 900, 1016, 1038, 889, 735, 694, 350, 149]

In [10]:
smape,real,previsto = calc_smape(resposta,exato)
        
os.makedirs("./imgteste", exist_ok=True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(real))), y=real, mode='lines', name='Valor Real'))
fig.add_trace(go.Scatter(x=list(range(len(previsto))), y=previsto, mode='lines', name='Valor Previsto'))

fig.update_layout(
    title=f'Passageiros por Horário - smape: {smape}%',
    xaxis_title='Horários',
    yaxis_title='Passageiros',
    showlegend=True,
    colorway=['#1f77b4', '#ff7f0e'],
    height=600,
)
fig.write_image(f"./imgteste/{linha}-TRANSFORMER.png", width=1400, height=600)

print(f"Imagem gerada para a linha {linha}")

Imagem gerada para a linha 52
